
<img src="https://www2.gmu.edu/sites/all/modules/features/feature_core_theme/templates/resources/images/mason-logo.png " alt="GMU Logo" title="George Mason University" />
<hr style ="color:#99CC99">
    
<h2 style="font-family:Helvetica; color:#006633;">Programming Assignment # 2</h2>
<h3 style="font-family:Helvetica; color:#006633;"> Feed-forward Neural Network for Sentiment Classification and Language Modelling</h3>

<p style="font-family:Helvetica; font-size:1.5em;"> 
Authors: Team 1 - Shinoj Jerald Bounaventure Kumar Jeronmary, Yitong Li, Anh Nguyen and Nina Nnamani<br>
Course Professor: Dr. Lindi Liao <br>
Course Name: Natural Language Processing <br>
Course Name and Section#: AIT 726-001<br>
University Name: George Mason University<br>
Date: October 8, 2020    <br>
</p>    
<hr style ="color:#99CC99" width="75%">
<p style="font-family:Helvetica; font-size:1.4em;"> 
Description: In this assignment, you will build feed forward neural networks for sentiment classification and language modelling. We will use the same data from Assignment 1. You will have to build the system from the scratch (e.g. numpy). Do not use any existing libraries (e.g. scikit-learn, tensorflow). 
 <br>
    
<p style="font-family:Helvetica; font-size:1.4em;"> 
Instructions: ...<br>

<p style="font-family:Helvetica; font-size:1.2em;"> 
1) ...<br>
2) ... <br>

<br> 
           
<p style="font-family:Helvetica; font-size:1.4em;"> 
References: The following listed sources provided some insight on code techniques that were partially adapted.

Build the feed forward neural network
https://towardsdatascience.com/how-to-build-your-own-neural-network-from-scratch-in-python-68998a08e4f6
    

 <br>

<hr style ="color:#99CC99">


## Import Libraries

In [2]:
#Import packages
import os
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import collections
import re
import numpy as np
import time
import xml
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")

#captures runtime
start_time = time.time()


## Feed forward neural network for sentiment classification

## Nina and Anh

### a) Create your Vocabulary: Read the complete training data word by word and create the vocabulary V for the corpus. You must not include the test set in this process. Remove any markup tags, e.g., HTML tags, from the data. Lower case capitalized words (i.e., starts with a capital letter) but not all capital words (e.g., USA). Remove all stopwords. You can use appropriate tools in nltk to stem. Stem at white space and also at each punctuation. In other words, “child’s” consists of two tokens “child and ‘s”, “home.” consists of two tokens “home” and “.”. Consider emoticons in this process. You can use an emoticon tokenizer, if you so choose. If yes, specify which one.

#Nina: copy/paste and test from homework1

Removes mark-up, normalizes capitalized first letter and removed emoticons/emojis


In [3]:
#reading the text data
pos_train = os.listdir("train/positive/")
neg_train = os.listdir("train/negative/")
pos_test = os.listdir("test/positive/")
neg_test = os.listdir("test/negative/")

In [4]:
#Removing Emoticons
def remove_emoji(string):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251" 
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030""]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string) # no emoji

In [5]:
#Removing HTML tags
def remove_tags(text):
    s = re.sub(r'<[^>]+>', '', text)
    return s

Creates vocabulary by incorporating function and tokenizing text for unstemmed and extract features for Bag of Words.

In [6]:
# Cleaning and adding the data
def create_list(dir,type,type1):
    return_list=[]
    for i in range(0, 500):
        file1 = open(type+"/"+type1+"/" + dir[i])
        try:
            text = file1.read()
            text = remove_tags(text)
            text = remove_emoji(text)
            return_list.append(text)
        except UnicodeDecodeError:
            k=0
        file1.close()
    return return_list

In [7]:
#Creating dataframe with positive and negative tweets
def createDataFrame(pos_train_list,neg_train_list):
    df1 = pd.DataFrame(neg_train_list)
    target2 = [0] * len(neg_train_list)
    df1["target"] = target2
    df1 = df1.rename(columns={0: "text"})
#Giving positive tweets as 1 and negative tweets as 0
    df = pd.DataFrame(pos_train_list)
    target1 = [1] * len(pos_train_list)
    df["target"] = target1
    df = df.rename(columns={0: "text"})
    
#Data is getting shuffled here
    data = pd.concat([df, df1])
    #data = data.sample(frac=1)
    x = list(data["text"])
    y=np.array(data["target"])

    return x,y;

In [9]:
# Tokenizing the text
def clean(text):

    vocab=[]
    for j in word_tokenize(text):
        if (j != ''):
            if not j.islower() and not j.isupper():
                j = j.lower()
            vocab.append(j)

    return vocab

In [10]:
# Tokenizing the text and stemming it
def cleaningStemmed(text):

    ps = PorterStemmer()
    vocabulary_stemmed=[]
    for j in word_tokenize(text):
        if (j != ''):
            if not j.islower() and not j.isupper():
                j = j.lower()
            vocabulary_stemmed.append(ps.stem(j))

    return vocabulary_stemmed

Combine functions
- load in the data
- create list
- make dataframe

In [11]:
#Combine functions to load training set 
pos_train_list = create_list(pos_train,"train", "positive")
neg_train_list = create_list(neg_train,"train", "negative")
X,y=createDataFrame(pos_train_list,neg_train_list)

In [14]:
pos_train_list

['@SouthwestAir I would appreciate that.  Thank you.\n',
 '@USAirways thank you very much.\n',
 "@JetBlue I'm all set. About to fly. Not bad for a first date with a giant metal bird machine. She even brought snacks.\n",
 '@SouthwestAir I got a flight at 11:55am on Thursday but looking for something tomorrow anything available?\n',
 "@AmericanAir you're my early frontrunner for best airline! #oscars2016\n",
 '@SouthwestAir #RedCarpet Southwest Companion Pass would be great!\n',
 "@USAirways @AmericanAir major issues getting out of Boston but your crew has been exceptional. Let's see how things roll out in Philly.\n",
 '@JetBlue thanks. I appreciate your prompt response.\n',
 '@united such a relaxing space for a drink before my flight! (at @United Global First Lounge) https://t.co/j4cj0lrF2d http://t.co/dTLGUQ1kAk\n',
 '@united ok. I just submitted. Thanks for the opportunity to give feedback.\n',
 '@united thank you\n',
 "@SouthwestAir was fantastic! That's the best flight service I've 

In [ ]:
#Checking Stemming/ Not Stemming data
for i in range(0, len(X)):
    X[i] = cleaningStemmed(X[i])

for i in range(0, len(X)):
    X[i] = clean(X[i])

# Creating vocabulary list
vocab = X[0]
  for i in range(1, len(X)):
  vocab.extend(X[i])
vocab = sorted(set(vocab))

    row = len(X)
    col = len(vocab)

    dict_vocab = {}
    for i, j in enumerate(vocab):
        dict_vocab[j] = i
    trainVector = np.zeros((row, col), dtype=np.int64)

IndentationError: ignored

### b) Extract Features: Convert documents to vectors using tf-idf representation. Do not use the existing tf-idf calculation library.

#Nina: copy/paste and test from homework1

In [15]:
# Vectorizing based user input
def vectorizer(X,vectorArr,dict_vocab,vectorType,row,col):

    if (vectorType==1):
        for i in range(0, len(X)):
            for j in X[i]:
                if j in dict_vocab:
                    vectorArr[i, dict_vocab[j]] += 1


        idf= np.zeros((row, col), dtype=np.int64)
        for i in range(0,len(vectorArr)):
            for j in range(0,col):
                if vectorArr[i][j] > 0:
                    idf[i][j]= math.log10(row / float(vectorArr[i][j]))

                else:
                    idf[i][j]=0
        vectorArr=np.multiply(vectorArr, idf)

    return vectorArr


# Vectorizing the trainset
trainVector=vectorizer(X,trainVector,dict_vocab,vectorType,row,col)
m, n = trainVector.shape
trainVector = np.concatenate([np.ones((m, 1)), trainVector], axis=1)

NameError: name 'trainVector' is not defined

### c) Training: Note that if you want to experiment with different stemmers or other aspects of the input features, you must do so on the training set, either through cross-validation or by setting a development set from the get-go. You must not do such preliminary evaluations on test data. Once you have finalized your system, you are ready to evaluate the test data. You can ignore any words that appear in the test set but not the training set.

# Anh(lead) and Nina(assist)

### d) Build and train a feed forward neural network: Build your FFNN with 2 layers with hidden vector size 20. Initialize the weights with random numbers. Use mean squared error as your loss function, sigmoid as the activation function. You can start training with a learning rate of 0.0001. If you would like to tune any parameters, you must do so using cross-validation on the training data only. Once you have finalized your system, you are ready to evaluate the test data.

#Nina(lead) and Anh(assist)

In [ ]:
#parameters
learning_rate = 0.0001
weights = np.random.rand(2,1)
bias = np.random.rand(1)


#activation function
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

#derivative of the sigmoid function
def sigmoid_derivative(x):
    return sigmoid(x)*(1-sigmoid(x)

### e) Evaluation: Compute the most likely class for each review in the test set using each of the combinations of stemming + tf-idf, no-stemming + tf-idf. Compute and report accuracy with confusion matrix on a .txt or .log file.

## Anh: Copy/paste and test from homework1

## Feed forward neural network for language modelling
## Yitong and Shinoj

### a) Pre-processing: Read the data word by word from a “train/positive”. Remove any markup tags, e.g., HTML tags, from the data. Lower case capitalized words (i.e., starts with a capital letter) but not all capital words (e.g., USA). Do not remove stopwords. Tokenize at white space and also at each punctuation. Consider emoticons in this process. You can use an emoticon tokenizer, if you so choose. If yes, specify which one.

## Copy/paste and test

In [16]:
import os
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import collections
import re
import numpy as np
import time
import xml
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")
import nltk
import random
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

#captures runtime
start_time = time.time()

def normalize_case(s):    
    '''
    Paramaeter: Word to be normalized
    Converts words with capitalized first letters in to lower case.
    '''
    if(not s.isupper()):
        return s.lower()
    else:
        return s

def remove_tags(s):
    '''
    Paramaeter: Word to be normalized
    Removes HTML tags
    '''
    s = re.sub(r'[^\w\s]', '', s)
    return s

def Remove(duplicate): 
    '''
    Calculate the unique words in a list
    '''
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

def count_words(rootdir):
    '''
    Parameter: root directory

    The funtion collects the training files. Tokenizes text into words. Creates stemmed vocabulary and
    Counts the the occurance of each word in each class(positve and negative).
    '''
    vocab = []

    bigram = []
    cleaned_document = []
    # For each directory in the path
    z = 0
    for subdir, dirs, files in os.walk(rootdir):
        # For each file in the directory
        for file in files:
            z = z + 1
            if (z > 1180):
                break
            cleaned_document = []
            f = open(rootdir + file, 'r',encoding='utf-8')  # use the absolute URL of the file
            lines = f.readlines()
            # For each line in the file
            for line in lines:
                document = word_tokenize(line)
                for i in range(0, len(document)):
                    # Normalize case for the word
                    document[i] = normalize_case(document[i])
                    # Remove HTML tags
                    document[i] = remove_tags(document[i])
                    if (document[i] != ''):
                        cleaned_document.append(document[i])
                        vocab.append(document[i])

            # Store as bigrams
            bigram.extend(list(nltk.bigrams(cleaned_document)))
    
    uni_vocab=Remove(vocab)        
    # return positive bigram and vocabulary
    return bigram, uni_vocab

### b) Construct your n-grams: Create positive n-gram samples by collecting all pairs of adjacent tokens. Create 2 negative samples for each positive sample by keeping the first word the same as the positive sample, but randomly sampling the rest of the corpus for the second word. The second word can be any word in the corpus except for the first word itself.

#Shinoj

In [17]:
def make_bigrams(rootdir):
    '''
    The function collects positive bigrams, creates negative bigrams from them
    and stores them in a csv file
    '''
    pos_bigram, vocab = count_words(rootdir)

    # Randomly creates 2 negative bigrams for each positive bigrams
    neg_bigram = []
    for bigram in pos_bigram:
        rand_neg = (bigram[0], random.choice(vocab))
        neg_bigram.append(rand_neg)
        rand_neg = (bigram[0], random.choice(vocab))
        neg_bigram.append(rand_neg)

    # Combines the bigrams and stores it in a csv
    pos_df = pd.DataFrame(pos_bigram, columns=['first_word', 'second_word'])
    pos_df['tag'] = "pos"

    neg_df = pd.DataFrame(neg_bigram, columns=['first_word', 'second_word'])
    neg_df['tag'] = "neg"

    df = pos_df.append(neg_df, ignore_index=True)
    
    return df

### c) Build and train a feed forward neural network: Build your FFNN with 2 layers with hidden vector size 20. Initialize the weights with random numbers. Use mean squared error as your loss function, sigmoid as the activation function. You can start training with a learning rate of 0.0001 or 0.00001. If you would like to tune any parameters, you must do so using cross-validation on the training data only. Once you have finalized your system, you are ready to evaluate the test data

## Yitong and Shinoj

In [24]:
# bigram with labels
df=make_bigrams("./train/positive/")

# Extracting Input and Output variables
y = df['tag']
X = df['first_word'] + ' ' + df['second_word']


In [33]:
y

0        pos
1        pos
2        pos
3        pos
4        pos
        ... 
47077    neg
47078    neg
47079    neg
47080    neg
47081    neg
Name: tag, Length: 47082, dtype: object

In [21]:
# Converting text data into vectorized format(Count Vectorizer)
X1 = CountVectorizer()
X1.fit(X)
data = X1.transform(X)
data_array = data.toarray() # input

# Label Encoding the data
le = preprocessing.LabelEncoder()
le.fit(y)
Y = le.transform(y) # labeled output
labels=Y.reshape(data_array.shape[0],1)

# Feed forward neural network model
# the biases are ignored in this code

hidden_size = 20 # hidden layer size
np.random.seed(42)
weights1 = np.random.rand(data_array.shape[1],hidden_size)
weights2 = np.random.rand(hidden_size,1) 
lr = 0.00001
train, test, y_train, y_test = train_test_split(data_array, labels, test_size=0.3, random_state=1000)
labels=y_train

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_der(x):
    return sigmoid(x)*(1-sigmoid(x))

for epoch in range(200): # recommend setting epoch to 10 to quickly run the code. Setting epoch to 2000 when training the weight
    inputs = train

    # feedforward layer 1
    layer1 = sigmoid(np.dot(inputs, weights1))
    # feedforward output layer
    output = sigmoid(np.dot(layer1, weights2))
    
    # use 0.5 as a threshold. If less than 0.5, assigns label 0; if more than 0.5, assigns label 1
    for i in range(0,len(output)):
        if (output[i] <= 0.5):
            output[i] = 0
        else:
            output[i] = 1

    # backpropagation 
    # calculate errors
    # error = output - labels
    # print(error.sum())
    
    # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
    # the derivative mse (1/n)(labels - output)^2 is 2/n [(labels - output)]. Here, 2/n is constant and therefore can be ignored
    weights2_delta = (output - labels) * sigmoid_der(output)   
    weights1_delta = np.dot((output - labels) * sigmoid_der(output), weights2.T) * sigmoid_der(layer1)    
    
    # update weights1 and weights2
    d_weights2 = np.dot(layer1.T, weights2_delta) 
    d_weights1 = np.dot(inputs.T, weights1_delta)
    
    weights1 -= lr * d_weights1
    weights2 -= lr * d_weights2

### d) Evaluate: Compute the most likely class for each n-gram in the test set. Use the test set in the “test/positive” folders. Save your results in a.txt or .log file.

## Yitong

In [22]:
# evaluate
# use the trained model to predict
layer1 = sigmoid(np.dot(test, weights1))
output = sigmoid(np.dot(layer1, weights2))
# use 0.5 as a threshold. If less than 0.5, assigns label 0; if more than 0.5, assigns label 1
for i in range(0,len(output)):
    if (output[i] <= 0.5):
        output[i] = 0
    else:
        output[i] = 1

### e) Evaluation Methods: Compute and report accuracy. Save your output and accuracy in a .txt or .log file.

## Yitong


In [23]:
error = output - y_test # is error is non zero, then it is not accurately predicted
z=0
for i in range(0,len(error)):
    if (error[i] == 0):
        z=z+1

accuracy = z/len(test)
print(accuracy)

0.36658407079646016
